In [20]:
data = '''
%jx -> rt, rs
&cc -> cd, fc, qr, nl, gk, zr
%qs -> cl, rs
%zr -> cq
%mx -> nr, pm
%mj -> qr, cc
%cj -> cc, nt
%jv -> sp
%dj -> bd, zc
%kt -> lt
broadcaster -> gz, xg, cd, sg
&dn -> rx
%br -> nf, bd
%cd -> cc, nl
%zc -> jq, bd
%xg -> cf, pm
%nz -> gm, bd
&dd -> dn
%nb -> sl
&pm -> kt, xg, xp, jv, sp
&fh -> dn
%rt -> qq
%qq -> rs, hd
%hd -> qs, rs
&xp -> dn
%pj -> cc, mj
%gz -> bd, kb
%zd -> jv, pm
%cq -> cj, cc
%qr -> gk
%ng -> jk, bd
%kb -> bd, sv
%cl -> zx, rs
%gj -> zd, pm
%sl -> kx
%sv -> br
%nf -> bd, nz
%zx -> rs
%nt -> mn, cc
%rh -> nb, rs
%gk -> ln
&bd -> gm, gz, fh, sv
%jq -> ng, bd
%sp -> pc
%sg -> rs, rh
%kx -> jx
&fc -> dn
%cf -> gj, pm
%pc -> kt, pm
%jk -> bd
%vf -> pm
&rs -> sg, dd, sl, kx, nb, rt
%nr -> vf, pm
%ln -> zr, cc
%lt -> pm, mx
%gm -> dj
%nl -> pj
%mn -> cc
'''

In [29]:
# %: Flip-flop: on/off, init: off
# got high pulse: ignore
# got low: 
#     off -> on  -> send high
#     on  -> off -> send low

# &: Conjunction: remember last pulse of connected input modules
# default: low for each input
# got purse:
#     update input info ->
#         if all high -> send low
#         else -> send high

# broadcast: forward same pulse to all destination modules, button -> send low

# input_d = '''
# broadcaster -> a, b, c
# %a -> b
# %b -> c
# %c -> inv
# &inv -> a
# '''

# input_d = '''
# broadcaster -> a
# %a -> inv, con
# &inv -> b
# %b -> con
# &con -> output
# '''

def get_next_conjuction_pulse(conjuction_module, flip_flop, conjunction_input):
    is_all_high = all([pulse == 'h' for pulse in list(conjunction_input[conjuction_module].values())])
    return 'l' if is_all_high else 'h'

def get_next_flip_flop_pulse(is_original_status_on):
    return 'l' if is_original_status_on else 'h'

# todo: ex2: output
def parse_input(input_d, broadcaster, flip_flop, conjunction_output):
    for line in input_d.strip().split("\n"):
        (left, right) = line.split(" -> ")
        if left == "broadcaster":
            [broadcaster.append(e) for e in right.split(", ")]
        elif left[0] == '%':
            flip_flop[left[1:]] = (right.split(", "), 'l', False)
        elif left[0] == '&':
            conjunction_output[left[1:]] = (right.split(", "), 'l')
        else:
            raise Exception('invalid line: ', line)
    return

def build_conjunction_input(flip_flop, conjunction_output, conjunction_input):
    conjunction_output_keys = list(conjunction_output.keys())
    for key in conjunction_output_keys:
        conjunction_input[key] = {}
    for entry in flip_flop.items():
        for dest in entry[1][0]:
            if dest in conjunction_output_keys:
                conjunction_input[dest].update({entry[0]: 'l'})
    return
    
# [target1, target2 ...]
broadcaster = []
# ff value: dest, h/l, is_on
# {ff1: ([t1, t2], 'l', False)}
flip_flop = {}
# con output value: dest, h/l
# {c1: [t1, t2], 'h'}
conjunction_output = {}
# con input value: input module to pulse map
# {c1: {i1: 'h', i2: 'l'}}
conjunction_input = {}

parse_input(input_d, broadcaster, flip_flop, conjunction_output)
build_conjunction_input(flip_flop, conjunction_output, conjunction_input)

count_h = 0
count_l = 0

for round in range(1000):
    tasks = []
    tasks.append(('button', 'l', 'broadcaster'))
    
    while_loop = 0
    while len(tasks) > 0:
        while_loop += 1
        if while_loop > 999:
            raise Exception(while_loop)
        current_task = tasks.pop(0)
        # task 2 starts
        # &dd -> dn
        # &fh -> dn
        # &xp -> dn
        # &fc -> dn
        # if current_task[2] == 'dn' and current_task[0] == 'dd' and current_task[1] == 'h':
        # if current_task[2] == 'dn' and current_task[0] == 'fh' and current_task[1] == 'h':
        # if current_task[2] == 'dn' and current_task[0] == 'xp' and current_task[1] == 'h':
        # if current_task[2] == 'dn' and current_task[0] == 'fc' and current_task[1] == 'h':
        #     print(round + 1, current_task)
        # tast 2 ends
        
        if current_task[1] == 'h':
            count_h += 1
        else:
            count_l += 1
        # broadcaster
        if current_task[2] == 'broadcaster':
            next_pulse = 'l'
            [tasks.append((current_task[2], next_pulse, e)) for e in broadcaster]
    
        # flip flop
        flip_flop_instance = flip_flop.get(current_task[2], None)
        if flip_flop_instance is not None:
            # high is ignored
            if current_task[1] == 'l':
                ( dest, pulse, is_on ) = flip_flop_instance
                next_pulse = get_next_flip_flop_pulse(is_on)
                is_on = not is_on
                # update current pulse & is_on
                flip_flop.update({current_task[2]: (dest, current_task[1], is_on)})
                # next task
                [tasks.append((current_task[2], next_pulse, e)) for e in dest]
        # conjuction
        conjuction_output_instance = conjunction_output.get(current_task[2], None)
        if conjuction_output_instance is not None:
            ( dest, pulse ) = conjuction_output_instance
            # update input info
            conjunction_input[current_task[2]].update({current_task[0]: current_task[1]})
            input_modules = conjunction_input.get(current_task[2])
            # next purse
            next_pulse = get_next_conjuction_pulse(current_task[2], flip_flop, conjunction_input)
            # update current pulse
            conjunction_output.update({current_task[2]: (dest, current_task[1])})
            # next task
            [tasks.append((current_task[2], next_pulse, e)) for e in dest]
        # outcome
        else:
            continue
print(count_h*count_l)

899848294


In [27]:
print(4003*4027*3919*3917)

247454898168563
